In [1]:
from __future__ import print_function

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.layers import Dense, Input, Flatten
from keras.models import Sequential, Model
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image

import os

%matplotlib inline

Using TensorFlow backend.


In [2]:
IMAGE_DIRECTORY_PATH = "./ships-in-satellite-imagery/shipsnet/"

In [3]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()
K.set_session(sess)

In [4]:
# import h5py
# f = h5py.File("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5", "r")

In [5]:
file_list = os.listdir(IMAGE_DIRECTORY_PATH)

# Define number of examples
m = len(file_list)

In [7]:
image_data = np.zeros((m, 80, 80, 3))
label_data = np.zeros((m, 1, 1, 1))

for i in range(m):
    file_name = file_list[i]
    img = plt.imread(IMAGE_DIRECTORY_PATH + file_name, "r")
    image_data[i,:,:,:] = img
    label_data[i,:,:,:] = int(file_name.split("_", 1)[0])

In [14]:
X_train = image_data[:2500,:,:,:]
Y_train = label_data[:2500,:,:,:]

X_test = image_data[2500:,:,:,:]
Y_test = label_data[2500:,:,:,:]

In [15]:
input_tensor = tf.Variable(np.zeros((m, 80, 80, 3)),
                           dtype='float32', name="input_tensor")

In [16]:
model = vgg19.VGG19(weights=None,
#                     input_tensor=input_tensor,
                    include_top=False)
model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [25]:
dense_outputs = Dense(1, activation='softmax')(model.output)
prediction_model = Model(inputs=model.input, outputs=dense_outputs)

In [26]:
# prediction_model.summary()
prediction_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
prediction_model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
 544/2500 [=====>........................] - ETA: 23:31 - loss: 13.2756 - acc: 0.1673